In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
import os
import sys
sys.path.append('wma_pyTools')
startDir=os.getcwd()
print(startDir)
print(sys.path)
# import wmaPyTools.roiTools

/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment
['/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment', '/home/naxos2-raid9/hbraun/dbs/dbs-pipeline/src', '/opt/local/dbs/bin/DBS-segmentations-0.1.2', '/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment', '/opt/local/dbs/bin/miniconda3/envs/subsegment/lib/python310.zip', '/opt/local/dbs/bin/miniconda3/envs/subsegment/lib/python3.10', '/opt/local/dbs/bin/miniconda3/envs/subsegment/lib/python3.10/lib-dynload', '', '/opt/local/dbs/bin/miniconda3/envs/subsegment/lib/python3.10/site-packages', '/opt/local/dbs/bin/miniconda3/envs/subsegment/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg', '/opt/local/dbs/bin/miniconda3/envs/subsegment/lib/python3.10/site-packages/PyYAML-6.0-py3.10-linux-x86_64.egg', 'wma_pyTools', 'wma_pyTools']


In [6]:
import os
import sys
import itertools
import numpy as np
from pathlib import Path
import nibabel as nib
#import random

#make sure that wma_pyTools is right in the working directory, or that
#the package can otherwise be imported effectively
sys.path.append('wma_pyTools')
startDir=Path(os.getcwd())
import pandas as pd
import wmaPyTools.roiTools
import wmaPyTools.analysisTools
import wmaPyTools.segmentationTools
import wmaPyTools.streamlineTools
import wmaPyTools.visTools

#dipy
from dipy.tracking.utils import reduce_labels
from dipy.tracking import utils
import dipy.io.streamline
from dipy.tracking.utils import density_map

In [7]:
targetLabels={'left':[1002,1026,1012,1020,1028,1003,1014,1019,1027],
              'right':[2002,2026,2012,2020,2028,2003,2014,2019,2027]}
spineLabels = {'left': [28, 16, 10], 
               'right': [16, 60, 49]}

#paths to input data
data_dir = startDir /'indata'
track_files = {
    'left': [data_dir / 'inferior_aLIC_left.tck', data_dir / 'superior_aLIC_left.tck'],
    'right': [data_dir / 'inferior_aLIC_right.tck', data_dir / 'superior_aLIC_right.tck']}
parcellationPath=data_dir / 'aparc+aseg.nii.gz'
refT1Path=data_dir / 'T1w_acpc_dc_restore.nii.gz'

# Freesurfer lookup table, e.g. https://surfer.nmr.mgh.harvard.edu/fswiki/FsTutorial/AnatomicalROI/FreeSurferColorLUT
lutPath=data_dir / 'FreesurferLookup.csv'

#tckPath=Path('/home/naxos2-raid25/sreta001/DBS_for_sreta001/DBS-OCD/OCD004/Code/app-track_aLIC_harelpreproc/output/track.tck')
saveFigDir=startDir / 'output'


In [10]:
# sanity check inputs
to_check = [ parcellationPath, refT1Path, lutPath]
for side in ['left', 'right']:
    for tck in track_files[side]:
        to_check.append(tck)
        
for i in to_check:
    print(i)
    assert(i.is_file())

/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/aparc+aseg.nii.gz
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/T1w_acpc_dc_restore.nii.gz
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/FreesurferLookup.csv
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/inferior_aLIC_left.tck
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/superior_aLIC_left.tck
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/inferior_aLIC_right.tck
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/superior_aLIC_right.tck


In [11]:
# define functions to generate tck for target

def get_streams_matching_target(streams, atlas, target):
    target_mask=wmaPyTools.roiTools.multiROIrequestToMask(atlas,target)
    # return boolean mask for stream selection
    return wmaPyTools.segmentationTools.segmentTractMultiROI(streams, 
                    [target_mask,], 
                    [True,], 
                    ['either_end',]) 
    
def save_density_map(streams, ref_img, out_file):
    density=utils.density_map(streams, ref_img.affine, ref_img.shape)
    densityNifti = nib.nifti1.Nifti1Image(density, ref_img.affine, ref_img.header)
    nib.save(densityNifti, out_file)
    
def save_streams_matching_target(streams, atlas, lookupTable, target, out_file):
    strTarget = lookupTable.loc[target, 'LabelName:']
    print('target label is: %s (%s)' % (target, strTarget))
    #out_file = Path(save_dir) / ('track_%04d_%s' % (target,strTarget)) #no file extension yet, add it later
    print(out_file)
    # get boolean vector of matching streams
    targetBool = get_streams_matching_target(streams, atlas, target)
    streams = streams[targetBool]
    
    #dipy quickbundles
    streams = streams[bundle(streams)]
    
    #save *.tck tractogram
    wmaPyTools.streamlineTools.stubbornSaveTractogram(streams,
        savePath=str(out_file.with_suffix('.tck')))
    # save nifti density map
    save_density_map(streams, atlas, out_file.with_suffix('.nii.gz'))
    return targetBool
    
#targetBool = save_streams_matching_target(streams,inflatedAtlas, lookupTable, iTarget, saveFigDir)
#wmaPyTools.streamlineTools.stubbornSaveTractogram(streams[targetBool], 
#    savePath=str(saveFigDir / '1002_test.tck' )

In [12]:
#apply the initial culling, to remove extraneous streamlines 
#first requires doing a DIPY quickbundling
def bundle(streams):
    print("DIPY quickbundle")
    clusters=wmaPyTools.streamlineTools.quickbundlesClusters(streams, thresholds = [30,20,10], nb_pts=100)

    #use those clusters to identify the streamlines to be culled
    print("identify streamlines to remove")
    survivingStreamsIndices, culledStreamIndicies=wmaPyTools.streamlineTools.cullViaClusters(clusters,streams,3)
    #convert survivingStreamsIndicies into a bool vec
    survivingStreamsBoolVec=np.zeros(len(streams),dtype=bool)
    survivingStreamsBoolVec[survivingStreamsIndices]=True
    
    print('%d of %d streams survived' % (len(survivingStreamsIndices), len(survivingStreamsBoolVec)))
    
    return survivingStreamsBoolVec

In [13]:
# load atlas-baced segmentation (Dan calls it a parcellation)
parcellaton=nib.load(parcellationPath)

In [14]:
# load T1 anatomical image
refT1=nib.load(refT1Path)

In [15]:
# load Freesurfer labels
lookupTable=pd.read_csv(lutPath,index_col='#No.')

In [17]:
#perform inflate & deIsland of input parcellation
inflated_atlas_file = saveFigDir / Path(Path(parcellationPath.stem).stem + '_inflated').with_suffix('.nii.gz')
print(inflated_atlas_file)
inflatedAtlas,deIslandReport,inflationReport= wmaPyTools.roiTools.preProcParc(parcellaton,deIslandBool=True,inflateIter=2,retainOrigBorders=False,maintainIslandsLabels=None,erodeLabels=[2,41])    
nib.save(inflatedAtlas,filename=inflated_atlas_file)

/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/aparc+aseg_inflated.nii.gz
1034052 labels changed from 2 to 0
962710 labels changed from 41 to 0
inflation iteration 1


100%|███████████████████████████████| 1132905/1132905 [02:09<00:00, 8764.38it/s]


inflation iteration 2


100%|███████████████████████████████| 1124385/1124385 [02:07<00:00, 8793.44it/s]


In [18]:
#Main cell, do all the hard work

for iSide in ['left', 'right']:
    for track_file in track_files[iSide]:        
        # load & orient streamlines
        
        tck_oriented_file = saveFigDir / Path(track_file.stem + '_oriented').with_suffix('.tck')
        if tck_oriented_file.exists():
            print('oriented tck already exists. loading %s' % tck_oriented_file)
            tckIn=nib.streamlines.load(tck_oriented_file)
            streams = tckIn.streamlines
        else:
            print('Load tck %s' % track_file)
            tckIn=nib.streamlines.load(track_file)
            print("orienting streamlines")
            streams=wmaPyTools.streamlineTools.orientAllStreamlines(tckIn.streamlines)
            # do quickbundles (never mind, takes too long)
            #streams = streams[bundle(streams)]
            # save oriented + bundled streams
            print('saving oriented tck %s' % tck_oriented_file)
            wmaPyTools.streamlineTools.stubbornSaveTractogram(streams,savePath=str(tck_oriented_file))
        
        parent_density_file = saveFigDir / Path(track_file.stem).with_suffix('.nii.gz')
        print('saving density map %s' % parent_density_file)
        save_density_map(streams, inflatedAtlas, parent_density_file)
        
        for iTarget in targetLabels[iSide]:
            targetStr = lookupTable.loc[iTarget, 'LabelName:']
            out_file = saveFigDir / ('%s_%04d_%s' % (track_file.stem, iTarget, targetStr))
            print('Starting processing for %s' % out_file.stem)
            
            # subsegment the streams and save the resulting density map and tck tractogram
            targetBool = save_streams_matching_target(streams, inflatedAtlas, lookupTable, iTarget, out_file)

            

Load tck /home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/indata/inferior_aLIC_left.tck
orienting streamlines


100%|██████████████████████████████████| 210884/210884 [03:52<00:00, 907.60it/s]


104912 of 210884 streamlines flipped
saving oriented tck /home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/inferior_aLIC_left_oriented.tck
saving density map /home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/inferior_aLIC_left.nii.gz
Starting processing for inferior_aLIC_left_1002_ctx-lh-caudalanteriorcingulate
target label is: 1002 (ctx-lh-caudalanteriorcingulate)
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/inferior_aLIC_left_1002_ctx-lh-caudalanteriorcingulate
volumes of input ROIs to check [5934.]
volumes of output ROIs to check [5934]
   roiVolumes  inclusionCriteria  operations
0        5934               True  either_end


/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/wma_pyTools/wmaPyTools/segmentationTools.py:830: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  tractMask=nib.Nifti1Image(np.greater(tractMask,0).astype(int), affine=maskNifti.affine)


Tractogram subseting complete in 5.0481333769998855, 1120 of 210884 within mask boundaries
Tractogram segmentation complete in 0.14991869599998608, 390 of 210884 met INCLUSION criterion
DIPY quickbundle
identify streamlines to remove
382 of 390 streams survived
Starting processing for inferior_aLIC_left_1026_ctx-lh-rostralanteriorcingulate
target label is: 1026 (ctx-lh-rostralanteriorcingulate)
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/inferior_aLIC_left_1026_ctx-lh-rostralanteriorcingulate
volumes of input ROIs to check [10100.]
volumes of output ROIs to check [10100]
   roiVolumes  inclusionCriteria  operations
0       10100               True  either_end


...

Tractogram subseting complete in 7.26091213700056, 193 of 288087 within mask boundaries
Tractogram segmentation complete in 0.10635807100061356, 72 of 288087 met INCLUSION criterion
DIPY quickbundle
identify streamlines to remove
65 of 72 streams survived
Starting processing for superior_aLIC_right_2019_ctx-rh-parsorbitalis
target label is: 2019 (ctx-rh-parsorbitalis)
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/superior_aLIC_right_2019_ctx-rh-parsorbitalis
volumes of input ROIs to check [11144.]
volumes of output ROIs to check [11144]
   roiVolumes  inclusionCriteria  operations
0       11144               True  either_end


/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/wma_pyTools/wmaPyTools/segmentationTools.py:830: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  tractMask=nib.Nifti1Image(np.greater(tractMask,0).astype(int), affine=maskNifti.affine)


Tractogram subseting complete in 6.982288463000259, 5169 of 288087 within mask boundaries
Tractogram segmentation complete in 0.6063986799999839, 2154 of 288087 met INCLUSION criterion
DIPY quickbundle
identify streamlines to remove
2144 of 2154 streams survived
Starting processing for superior_aLIC_right_2027_ctx-rh-rostralmiddlefrontal
target label is: 2027 (ctx-rh-rostralmiddlefrontal)
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/superior_aLIC_right_2027_ctx-rh-rostralmiddlefrontal
volumes of input ROIs to check [70395.]
volumes of output ROIs to check [70395]
   roiVolumes  inclusionCriteria  operations
0       70395               True  either_end


/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/wma_pyTools/wmaPyTools/segmentationTools.py:830: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  tractMask=nib.Nifti1Image(np.greater(tractMask,0).astype(int), affine=maskNifti.affine)


Tractogram subseting complete in 7.340079041000536, 64230 of 288087 within mask boundaries
Tractogram segmentation complete in 28.74592285199924, 27899 of 288087 met INCLUSION criterion
DIPY quickbundle
identify streamlines to remove
27838 of 27899 streams survived


In [26]:
# generate combined niftis
for iSide in ['left', 'right']:
    for iTarget in targetLabels[iSide]:
        combined_img = np.zeros(refT1.shape)
        for track_file in track_files[iSide]:  # iterate over inferior and superior
            targetStr = lookupTable.loc[iTarget, 'LabelName:']
            in_file = saveFigDir / ('%s_%04d_%s' % (track_file.stem, iTarget, targetStr))
            print(in_file)
            combined_img += nib.load(in_file.with_suffix('.nii.gz')).get_fdata()
        combined_file  = saveFigDir / ('combined_aLIC_%04d_%s' % ( iTarget, targetStr))
        combinedNifti = nib.nifti1.Nifti1Image(combined_img, refT1.affine, refT1.header)
        print(combined_file)
        nib.save(combinedNifti, combined_file.with_suffix('.nii.gz'))
        

/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/inferior_aLIC_left_1002_ctx-lh-caudalanteriorcingulate
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/superior_aLIC_left_1002_ctx-lh-caudalanteriorcingulate
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/combined_aLIC_1002_ctx-lh-caudalanteriorcingulate
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/inferior_aLIC_left_1026_ctx-lh-rostralanteriorcingulate
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/superior_aLIC_left_1026_ctx-lh-rostralanteriorcingulate
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/combined_aLIC_1026_ctx-lh-rostralanteriorcingulate
/home/udall-raid2/DBS_collaborators/DBS_for_sreta001/DBS-OCD/SLEEP116V2/Code/subsegment/output/inferior_aLI

## Appendix

In [9]:
# load tractogram
# SKIPPED because we're iterating over multiple tractograms
tckIn=nib.streamlines.load(tckPath)

In [10]:
# orient all the steamlines, potentially not necessary given redundancy with 
# subsequent steps
# SKIPPED because we're using either_end selection

print("orienting streamlines")
orientedStreams=wmaPyTools.streamlineTools.orientAllStreamlines(tckIn.streamlines)

orienting streamlines


100%|████████████████████████████████| 2400000/2400000 [48:08<00:00, 830.78it/s]

1196452 of 2400000 streamlines flipped


In [11]:
# save oriented streams
# SKIPPED
print("save oriented tck")
subTckSavePath=os.path.join(saveFigDir,'track_oriented.tck')
wmaPyTools.streamlineTools.stubbornSaveTractogram(orientedStreams,savePath=subTckSavePath)


save oriented tck


In [25]:
# save lite streamlines
#SKIPPED
n_streams_to_keep = int(5E5)
select_bool = np.random.choice(range(len(orientedStreams)), n_streams_to_keep, replace=False)
lite_streams = orientedStreams[select_bool]
wmaPyTools.streamlineTools.stubbornSaveTractogram(lite_streams,
    savePath=str(Path(saveFigDir) / 'track_lite.tck'))


In [26]:
# select streamlines to work on

streams = lite_streams


In [46]:
from dipy.tracking.utils import density_map
from wmaPyTools.visTools import multiTileDensity

multiTileDensity(streams,refT1,saveFigDir,'density',densityThreshold=0,noEmpties=True)

/opt/local/dbs/bin/miniconda3/envs/subsegment/lib/python3.10/site-packages/nilearn/image/resampling.py:780: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(img, data, affine)


KeyboardInterrupt: 

In [49]:
M, grouping = utils.connectivity_matrix(streams, inflatedAtlas.affine, inflatedAtlas.get_fdata().astype(np.int),
                                        return_mapping=True,
                                        mapping_as_streamlines=True)

/tmp/ipykernel_23404/3961023847.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  M, grouping = utils.connectivity_matrix(streams, inflatedAtlas.affine, inflatedAtlas.get_fdata().astype(np.int),


In [67]:
np.shape(M)

(2036, 2036)

In [42]:
targets = targetLabels['left'] + targetLabels['right']
targets.sort()
for iTarget in targets:
    print(iTarget)
    print(lookupTable.loc[iTarget, 'LabelName:'])

1002
ctx-lh-caudalanteriorcingulate
1003
ctx-lh-caudalmiddlefrontal
1012
ctx-lh-lateralorbitofrontal
1014
ctx-lh-medialorbitofrontal
1019
ctx-lh-parsorbitalis
1020
ctx-lh-parstriangularis
1026
ctx-lh-rostralanteriorcingulate
1027
ctx-lh-rostralmiddlefrontal
1028
ctx-lh-superiorfrontal
2002
ctx-rh-caudalanteriorcingulate
2003
ctx-rh-caudalmiddlefrontal
2012
ctx-rh-lateralorbitofrontal
2014
ctx-rh-medialorbitofrontal
2019
ctx-rh-parsorbitalis
2020
ctx-rh-parstriangularis
2026
ctx-rh-rostralanteriorcingulate
2027
ctx-rh-rostralmiddlefrontal
2028
ctx-rh-superiorfrontal
